## Aquecendo


In [0]:
SELECT current_catalog(), current_schema()


In [0]:
USE CATALOG smart_claims_dev

In [0]:
SELECT current_catalog(), current_schema()


In [0]:
USE SCHEMA 01_bronze

In [0]:
DROP TABLE IF EXISTS claims

## EXPLORE THE DATA SOURCE FILES



Query os arquivos CSV usando o path do volume

In [0]:
SELECT *
FROM csv.`/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv` limit 10
--drop the table if it exists for demonstration

## Create a STREAMING TABLE using Databricks Auto Loader

- processa somente novos dados incrementalmente e eficientemente.
- tem suporte para python e SQL
- pode processar bilhões linhas
- auto loader é construído com Spark Structured Streaming

In [0]:
CREATE OR REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims
SCHEDULE EVERY 1 WEEK
AS
SELECT *
FROM read_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv',
  header => true
)



In [0]:
CREATE OR REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims
AS
SELECT *
FROM STREAM cloud_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv',
  header => true
)

In [0]:
-- This file defines a sample transformation.
-- Edit the sample below or add new transformations
-- using "+ Add" in the file browser.

CREATE OR REFRESH STREAMING LIVE TABLE smart_claims_dev.01_bronze.claims
COMMENT "Bronze — sinistros (claims) a partir de 00_landing/sql_server/claims.csv"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT
  *,
  _metadata.file_path               AS input_file,
  _metadata.file_modification_time  AS input_file_mtime,
  current_timestamp()               AS ingested_at
FROM cloud_files(
  "/Volumes/smart_claims_dev/00_landing/sql_server",
  "csv",
  map(
    "header", "true",
    "cloudFiles.inferColumnTypes", "true",
    "cloudFiles.schemaEvolutionMode", "addNewColumns",
    "pathGlobFilter", "claims.csv"
  )
);

In [0]:
SELECT * FROM smart_claims_dev.01_bronze.claims
LIMIT 10;

In [0]:
DESCRIBE TABLE EXTENDED smart_claims_dev.01_bronze.claims;

In [0]:
DROP TABLE IF EXISTS smart_claims_dev.`01_bronze`.claims

In [0]:
-- Vamos criar novamente a tabela sem inferir o schema

CREATE OR REFRESH STREAMING LIVE TABLE smart_claims_dev.01_bronze.claims
COMMENT "Bronze — sinistros (claims) a partir de 00_landing/sql_server/claims.csv"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT
  *,
  _metadata.file_path               AS input_file,
  _metadata.file_modification_time  AS input_file_mtime,
  current_timestamp()               AS ingested_at
FROM cloud_files(
  "/Volumes/smart_claims_dev/00_landing/sql_server",
  "csv",
  map(
    "header", "true",
    "cloudFiles.inferColumnTypes", "false",
    "cloudFiles.schemaEvolutionMode", "addNewColumns",
    "pathGlobFilter", "claims.csv"
  )
);

**Importante**

Schema Evolution Mode

![](/Workspace/Users/thiagovdferraz@gmail.com/trilha_databricks/aula04/image/schema_evolution_mode.png)

Descrever a tabela novamente

In [0]:
DESCRIBE TABLE EXTENDED smart_claims_dev.01_bronze.claims;

In [0]:
DESCRIBE HISTORY smart_claims_dev.01_bronze.claims;

In [0]:
SELECT * FROM smart_claims_dev.01_bronze.claims limit 10

# Data ingestion
## Importante saber!

![](/Workspace/Users/thiagovdferraz@gmail.com/trilha_databricks/aula04/image/data_ingestion.png)